# Lab 2 Point pattern analysis - exploring point process models
In this lab we will perform point pattern analysis using R. Similar to previous labs, we will use a set of libraries and then create charts as well as maps of our data.

## Point processes in *R*
The major strength of R is the wide range of packages that have been developed for performing specialized statistical analysis (like spatial analysis). For point patterns, the go-to package is `spatstat`, so we start by loading it.

In [ ]:
library(spatstat)
library(RColorBrewer)

The user guide for **`spatstat`** is here: http://www.stats.uwo.ca/faculty/kulperger/S9934a/Papers/SpatStatIntro.pdf

For our purposes, what you really need to know is that spatstat is the most powerful tool around for
doing point pattern analysis.
    
Consequently you can use spatstat to generate a wide variety of point patterns using a range of point
processes. Among these are the following R commands:
    
* `rpoint()` generates a specified number of random uniform distributed points
* `rpoispp()` generates random points distributed according to some specified intensity pattern (which can be uniform)
* `rThomas()` generates clustered points via a ‘parent-child’ process
* `rSSI()` generates points that exclude one another from being with some specified distance of each other

There are several others also available. You can see the full list in the general help page for the package in the subsection entitled **To simulate a random point pattern**.

### Making a point pattern
Before we dive into this more fully, you can make a point pattern by invoking one of the above commands, and assigning the result to a variable. A simple example is to use a simple random point pattern with 100 events, assigning the result to a variable called `point_pattern`.

In [ ]:
point_pattern <- rpoint(100)

We can ask to see what the resulting data are, and get a basic description

In [ ]:
point_pattern

And we can also plot the result, which shows us what it looks like.

In [ ]:
plot(point_pattern)

We will be using the above basic sequence of commands a lot in this lab, so make sure you understand it. Try changing the number of events in the pattern. Or try using one of the other point processes. For any process more complicated than `rpoint()` or `rpoispp()` you will need to specify more than one parameter. Take a look at the documentation to see if you can figure out what parameters to provide.

### Pre-cooked built in point patterns
`spatstat` also provides some simple point pattern datasets, which we'll use to explore ways to visualize point patterns (particularly density estimation). The built in datasets are `redwoodfull`, `japanesepines` and `cells`. We access them by using the `data` command

The names are annoying to type so we can assign them to different variables for convenience

In [ ]:
rw <- redwoodfull
jp <- japanesepines
ce <- cells

We can plot any of these individually or request more information by typing its name, or using `summary()`. Try it:

In [ ]:
# Type commands to find out more about the three point patterns



We can also plot any of these using the `plot()` command. It is often useful in R to see a number of plots side-by-side, which we can do by issuing a `par()` command to change the *graphics parameters*. If we want one row of three plots side by side, we get this using

In [ ]:
par(mfrow=c(1,3))
plot(rw)
plot(ce)
plot(jp)

This looks a bit weird because of the overall dimensions of the plot space in notebook mode.  The default size for plots in R jupyter notebooks is 7 inches square, which we can change using an `options` command

In [ ]:
options(repr.plot.width=8, repr.plot.height=2.5)
par(mfrow=c(1,3))
plot(rw)
plot(ce)
plot(jp)

Finally, we can devote more space to the actual figures by making the margins in a plot narrower

In [ ]:
options(repr.plot.width=8, repr.plot.height=2.5)
par(mfrow=c(1,3), mai=c(0.1,0.1,0.1,0.1))
plot(rw)
plot(ce)
plot(jp)

You should experiment with the `options()` and `par()` settings to make sure you can control them so as to maximize the usefulness of any plots you make in the remainder of this lab.

### Visualizing the intensity (density) of a point pattern
An important way to improve our sense of a point pattern is to create density map of it. We can do this with the `density()` command. The sequence of commands below shows how to do this and overlay the point pattern itself.

In [ ]:
options(repr.plot.width=4, repr.plot.height=4) # this is a better size for a single pattern
plot(density(rw))
plot(rw, add=T)

The bandwidth is automatically chosen by R, but we can opt instead to set a bandwidth

In [ ]:
options(repr.plot.width=8, repr.plot.height=2.5)
par(mfrow=c(1,3), mai=rep(0.1,4))
plot(density(rw, sigma=0.05), main='bw=0.05')
plot(rw, add=T)
plot(density(rw, sigma=0.1), main='bw=0.1')
plot(rw, add=T)
plot(density(rw, sigma=0.25), main='bw=0.25')
plot(rw, add=T)


The `sigma` parameter controls the bandwidth used. Note that the dimensions of the region covered by the built in point patterns are a 1 by 1 square, and the bandwidth is expressed in the same units.  In cases where you are working with datasets that extend over 100s or 1000s or meters or miles (or whatever) you have to express the bandwidth required appropriately.

Experiment with settings in the previous cells until you feel confident with plotting density maps of point patterns.

### Nicer colors
The color scheme used by default for density maps is ... not great. We can change it out for a Color Brewer palette which may be preferable.

In [ ]:
par(mfrow=c(1,3), mai=rep(0.1,4))
plot(density(rw), col=brewer.pal(9, 'Reds'))
plot(rw, add=T)
plot(density(rw), col=brewer.pal(9, 'YlGnBu'))
plot(rw, add=T)
plot(density(rw), col=brewer.pal(9, 'Greys'))
plot(rw, add=T)


A final tweak to the appearance, is to make the events smaller, which can sometimes make things easier to see.

In [ ]:
par(mfrow=c(1,3), mai=rep(0.1,4), cex=0.5)
plot(density(rw), col=brewer.pal(9, 'Blues'))
plot(rw, add=T)
plot(density(rw), col=brewer.pal(9, 'RdBu'))
plot(rw, add=T)
plot(density(rw), col=brewer.pal(9, 'PRGn'))
plot(rw, add=T)

Make sure you have a good idea of how all these plotting options work before proceeding to the next section.

## The assignment proper: exploring point processes

The point (ahem) of this lab is to make some point patterns using `spatstat` commands, to examine them, and to comment on them.  To support this aim, you might find it useful to plot densities, with the points superimposed on top, as in the previous section.  To make the point patterns use the commands mentioned above and further explained below. You will also need to experiment to get the patterns to illustrate what you want to say.

The next sections explain some of the point process commands available in a bit more detail.

### The independent random process (or 'complete spatial randomness')
The `rpoint()` or `rpoispp()` commands will generate patterns with no first or second order effects such that there is no bias in where events occur and no interaction between events. `rpoint()` is simpler and will produce exactly the number of events requested. `rpoispp` is a *Poisson process* and produces events at the requested *intensity* per unit area. Since the default region for a point pattern in `spatstat` is a 1 by 1 unit square, this will seem very similar to `rpoint()` except that the number of events produced in a particular realization of `rpoispp()` will vary each time you run it.

In [ ]:
options(repr.plot.width=8, repr.plot.height=2.5)
par(mfrow=c(1,2), mai=rep(0.15,4))
p1 <- rpoint(100)
p2 <- rpoispp(100)
plot(p1)
plot(p2)

You can confirm these patterns have different numbers of events using the `summary()` command.

### Patterns with first order effects: the inhomogeneous Poisson process
If this was all you could do with `rpoispp()` it probably wouldn't justify it being a completely different point process than `rpoint()`. In fact, it can do a lot more. If instead of specifying a number, which produces a uniform intensity, you provide a function, then you can introduce spatial trends across the area occupied by the pattern.

In [ ]:
par(mfrow=c(1,3), mai=rep(0.1,4))
p1 <- rpoispp(function(x,y){200*x^2 + 200*y^2})
p2 <- rpoispp(function(x,y){200*(1-x)^2 + 200*y^2})
p3 <- rpoispp(function(x,y){400*(x-0.5)^2 + 400*(y-0.5)^2})
plot(p1, main='SW-NE trend')
plot(p2, main='NW-SE trend')
plot(p3, main='Increasing from the center')

In these patterns a fixed value for the process intensity is replaced with a function that converts an `x` and `y` value into a number according to some formula. For `p1` the function is defined by the code `function(x,y){200*x^2 + 200*y^2}` in other words, $200x^2 + 200y^2$. Since the default coordinates that the pattern is generated in are the unit square from $(0,0)$ to $(1,1)$ the pattern intensity ranges from 0 to 400 events per unit area with increasing intensity from low coordinates at the lower left to higher coordinates at the upper right.

To convince yourself that the trends in the titles of those plots are real, try mapping the density of the point pattern in each case, i.e. for patterns `p1`, `p2` and `p3`.

In [ ]:
## Write code to plot p1, p2 and p3 on top of maps showing their density


It's worth noting that you can also define the function for the intensity outside of the commande to make the point pattern&mdash;this is especially more convenient if the function is complex.  For example

In [ ]:
# I have no idea why you would have a slope like this but it shows the idea
complicated_slope <- function(x,y) {
    return(200 * (x * sin((y+0.5) * 2*pi/3) + y * cos((x-0.5) * 2*pi/3)))
}

In [ ]:
options(repr.plot.width=4, repr.plot.height=3.5)

p <- rpoispp(complicated_slope)
plot(density(p))
plot(p, add=T)

### Introducing second order effects
There are a number of point processes that will introduce second order (interaction) effects into a pattern. I will not explain them in detail, but suggest that you consult help https://www.rdocumentation.org/ for full details. First is a *simple sequential inhibition* process provided by the `rSSI()` command, which can simulate competition for space between point events and yields evenly spaced patterns.  Second is a 'parent-child' clustering process known as the *Thomas process* which the `rThomas()` command simulates.

You will find documentation for these and several other point processes supported by `spatstat` at the following pages, or linked from them:
- [rSSI](https://www.rdocumentation.org/packages/spatstat/versions/1.55-0/topics/rSSI)
- [rThomas](https://www.rdocumentation.org/packages/spatstat/versions/1.55-0/topics/rThomas)

The gist of it is:
- SSI has two parameters. The first is an inhibition distance within which new events are not placed closer to existing events, and the second is the number of events that the process will attempt to randomly place given the inhibition constraint.
- The Thomas process takes three parameters.  The first specifies the intensity of a Poisson process that generates ‘parent’ events, the second specifies a distance that ‘child’ events will be scattered away from their parents, and the third specifies the average number of children that each parent will produce.

Here are some example plots:

In [ ]:
options(repr.plot.width=8, repr.plot.height=2)
par(mfrow=c(1,4), mai=rep(0.1,4))
plot(rSSI(0.01, 100))
plot(rSSI(0.08, 100))
plot(rThomas(10, 0.1, 10))
plot(rThomas(25, 0.8, 4))

# The assignment
You need to do two things, as set out in the cells below.

## FIRST 
### Make three or four plots to show patterns with 
1. no first and second order effects
2. clear first order effects, and 
3. clear second order effects. 

**You will need to write code to do this.  The cells below are provided for completing each of these items. Snippets of code in the materials above shoule enable you to do this, without too much trouble.**

### 1. No first or second order effects

In [ ]:
# Write some code to make three or four plots

### 2. clear first order effects

In [ ]:
# Write some code to make three or four plots

### 3. clear second order effects

In [ ]:
# Write some code to make three or four plots

\[double-click to add any additional commentary you would like on the first part in this cell - note that no commentary is required\]

## SECOND 
### Make two plots where a point process generates a pattern that seems atypical for the process. 
**Of course, if you are able to make such a plot, then it would suggest the pattern is not atypical of the process!** 

What I mean is this: make plots of patterns that **you know** to have first or second order effects in operation, but where how those effects act produces patterns that don’t obviously exhibit those effects. 

For example, see if you can make the Thomas process produce patterns that don’t appear clustered, or if you can get realizations of a simple random process that appear to include clusters, or a trend.  

In [ ]:
# Write some code to produce your first 'contradictory' pattern

In [ ]:
# Write some code to produce another contradictory pattern

\[double-click to add dditional commentary you would like in this cell - for this part, commentary is required, to explain your understanding of what you've done\]

## Submission instructions
You should **File - Save and Checkpoint** the notebook when it is in a final state and ready for submission. 

You should then **File - Download as - PDF via LaTeX** a PDF file version for your personal records (this file will be a record of what you did), and **File - Download as - Notebook (.ipynb)** it as a .ipynb file and upload that file to the dropbox provided on bCourses.